# imports and initializations

In [4]:
# add our custom modules to the python import path
import sys
import os
sys.path.append(os.getcwd() + '/../../Client_modules/')

import mResSweep
from socProxy import makeProxy
import h5py
from PythonDrivers.control_atten import setatten
# import PythonDrivers.YOKOGS200 as YOKOGS200
%matplotlib qt
import matplotlib.pyplot as plt
savePath = r'Z:\t1Team\Data\TALE02_06_16_22\RFSOC\freqDomain'

# Python 3.7
# os.environ['PATH'] = os.getcwd() + '\\PythonDrivers' + os.pathsep + os.environ['PATH']
# Python 3.8 and newer
os.add_dll_directory(os.getcwd() + '/../../Client_modules/PythonDrivers')

<AddedDllDirectory('C:\\Users\\my\\Documents\\GitHub\\ZCU216_Russell\\res_dev\\notebooks\\2022-06-16 resonators/../../Client_modules/PythonDrivers')>

In [5]:
# create proxy to ZCU216 board
soc, soccfg = makeProxy('192.168.1.120')
print(soccfg)

Pyro.NameServer PYRO:Pyro.NameServer@192.168.1.120:8888
myqick PYRO:obj_032068d89f56432daba14108ce8239b6@192.168.1.120:43613

QICK configuration:

	Board: ZCU216

	Global clocks (MHz): tProcessor 349.997, RF reference 245.760

	7 signal generator channels:
	0:	axis_sg_int4_v1 - tProc output 0, switch ch 0, maxlen 4096
		DAC tile 2, ch 0, 16-bit DDS, fabric=430.080 MHz, fs=1720.320 MHz
	1:	axis_sg_int4_v1 - tProc output 1, switch ch 1, maxlen 4096
		DAC tile 2, ch 1, 16-bit DDS, fabric=430.080 MHz, fs=1720.320 MHz
	2:	axis_sg_int4_v1 - tProc output 2, switch ch 2, maxlen 4096
		DAC tile 2, ch 2, 16-bit DDS, fabric=430.080 MHz, fs=1720.320 MHz
	3:	axis_sg_int4_v1 - tProc output 3, switch ch 3, maxlen 4096
		DAC tile 2, ch 3, 16-bit DDS, fabric=430.080 MHz, fs=1720.320 MHz
	4:	axis_signal_gen_v6 - tProc output 4, switch ch 4, maxlen 65536
		DAC tile 3, ch 0, 32-bit DDS, fabric=599.040 MHz, fs=9584.640 MHz
	5:	axis_signal_gen_v6 - tProc output 5, switch ch 5, maxlen 65536
		DAC tile 3, ch 

# Define sweep parameters

In [9]:
inputDict = {}
inputDict['LO_f'] = 5e3  # local oscillator frequency

# for spectrum analyzer shots
# inputDict['mixerCenter_f'] = 1700  # center frequency for the mixer
# inputDict['span_f'] = 0  # frequency span to sweep over. The same for all resonators
# inputDict['res_f'] = [5.9e3, 6.411658e3, 7.05506e3, 7.5e3]  # center frequencies of the resonators

#frequencies for large span measurement
# inputDict['LO_f'] = 0e3  # local oscillator frequency
# inputDict['mixerCenter_f'] = 1.5e3  # center frequency for the mixer
# inputDict['span_f'] = 1000  # frequency span to sweep over. The same for all resonators
# inputDict['res_f'] = [1.5e3]  # center frequencies of the resonators

# define frequencies
# inputDict['mixerCenter_f'] = 1700  # center frequency for the mixer
# inputDict['span_f'] = .08  # frequency span to sweep over. The same for all resonators
# inputDict['res_f'] = [5.9e3, 6.401658e3, 7.04506e3, 7.5e3]  # center frequencies of the resonators

# define frequencies
inputDict['mixerCenter_f'] = 1700  # center frequency for the mixer
inputDict['span_f'] = .04  # frequency span to sweep over. The same for all resonators
inputDict['res_f'] = [6.1477131e3, 6.5812862e3]  # center frequencies of the resonators

# define numbers of points
inputDict['n_expts'] = 601  # number of points along the frequency axis

# timing. Units are clock cycles, but converted from us
inputDict['ring_up_time'] = 500  # time waiting for the resonator to ring up at the start of each sweep
inputDict['ring_between_time'] = 50  # time waiting for the resonator to ring up at the start of each sweep
inputDict['adc_trig_offset'] = 0.1 # time after the DAC starts the final steady pulse before the ADC starts it's read

# power
inputDict['basePower'] = 0
inputDict['attenSerial'] = [27712, 27784]
inputDict['gain'] = [1, 0.7]

# parameters which vary per power point
# inputDict['attenList'] = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
# inputDict['readout_lengthList'] = [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]  # time to average over
# inputDict['n_roundsList'] = [2, 2, 2, 2, 2, 4, 8, 10, 10, 10]  # number of times to sweep along the frequency axis
# inputDict['n_repsList'] = [200, 200, 200, 200, 200, 500, 1000, 2000, 2000, 4000] # number of repetitions to take at each frequency point

inputDict['attenList'] = [ 60, 70, 80, 90]
inputDict['readout_lengthList'] = [ 1000, 1000, 1000, 1000]  # time to average over
inputDict['n_roundsList'] = [4, 8, 10, 10]  # number of times to sweep along the frequency axis
inputDict['n_repsList'] = [500, 1000, 2000, 2000] # number of repetitions to take at each frequency point

# Acquire

In [10]:
# Acquire
for i, atten in enumerate(inputDict['attenList']):
    # update per power parameters
    inputDict['power'] = inputDict['basePower']-atten
    inputDict['n_rounds'] = inputDict['n_roundsList'][i]
    inputDict['n_reps'] = inputDict['n_repsList'][i]
    inputDict['readout_length'] = inputDict['readout_lengthList'][i]

    # set attenuation. Split it evenly across all attenuators
    # NOTE: What happens if we average to somewhere between the resolution of the attenuators? Possible issue
    for serial in inputDict['attenSerial']:
        setatten(atten/len(inputDict['attenSerial']), serial, printv=True)

    # run a frequency sweep
    Instance = mResSweep.ResSweep(path=savePath, prefix='data_p'+str(inputDict['power']), inputDict=inputDict, soc=soc, soccfg=soccfg)
    data = mResSweep.ResSweep.acquire(Instance)
    mResSweep.ResSweep.display(Instance, data)
    mResSweep.ResSweep.save_data(Instance, data)

Setting attenuation
4  device(s) found
Device 0 Serial number: 27784
Device 1 Serial number: 27713
Device 2 Serial number: 25630
Device 3 Serial number: 27712
Device was found to be device 3
Set attenuation: 30.0
Device 4 Serial number: 0
Setting attenuation
4  device(s) found
Device 0 Serial number: 27784
Device was found to be device 0
Set attenuation: 30.0
Device 1 Serial number: 27713
Device 2 Serial number: 25630
Device 3 Serial number: 27712
Device 4 Serial number: 0
Pulse frequencies are: [-552.28690023 -118.71380053] MHz
Frequencies out are: [1147.7131, 1581.2861999999996] MHz


C:\Users\my\Documents\GitHub\ZCU216_Russell\res_dev\notebooks\2022-06-16 resonators/../../Client_modules\mResSweep.py:256: UserWarning: Temperature log is 546.4 minutes out of date. Make sure you are outputting the log file from the temperature PC
  warnings.warn(


  0%|          | 0/601 [00:00<?, ?it/s]

Round 0, time 320.347 s


  0%|          | 0/601 [00:00<?, ?it/s]

...

Round 8, time 1221.853 s


  0%|          | 0/601 [00:00<?, ?it/s]

Round 9, time 1221.430 s
Final time = 12216.088 s


C:\Users\my\Documents\GitHub\ZCU216_Russell\res_dev\notebooks\2022-06-16 resonators/../../Client_modules\mResSweep.py:256: UserWarning: Temperature log is 1057.7 minutes out of date. Make sure you are outputting the log file from the temperature PC
  warnings.warn(


Saving data_p-90_017


In [ ]:
print(inputDict['gain'])

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(4.5, 4.5), dpi=75)

In [ ]:
axs